In [1]:
# using pandas to process data
from collections import Counter
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import pearsonr

## 1.数据载入
（将数据拼接起来：201701-201804)，并重新载入

In [2]:
# Step1:load data from .CSV and combine them together, 以日期作为索引
# Step1.1:load 'airQuality_201701-201801.csv'
airQuality_1 = pd.read_csv('data/airQuality_201701-201801.csv', parse_dates=['utc_time'], index_col=['stationId', 'utc_time'])
airQuality_1.columns = ['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']

# Step1.2:load 'airQuality_201802-201803.csv'
airQuality_2 = pd.read_csv('data/airQuality_201802-201803.csv', parse_dates=['utc_time'], index_col=['stationId', 'utc_time'])
airQuality_2.columns = ['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']

# Step1.3:load 'aiqQuality_201804.csv'
airQuality_3 = pd.read_csv('data/aiqQuality_201804.csv', parse_dates=['time'], index_col=False)
del airQuality_3['id']
airQuality_3.columns = ['stationId', 'utc_time','PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']
airQuality_3.set_index(['stationId', 'utc_time'], inplace=True)
airQuality_3.head()
# Step1.4:concat them together
airQuality = pd.concat([airQuality_1, airQuality_2,airQuality_3])
airQuality.to_csv('data/airQuality.csv')

In [10]:
# step2: 将location_aq_PM等特征列拼接起来
def load_bj_aq_data(aq_df):

    '''
    aq_df : a dataframe after concat.
    '''
    aq_dataset = aq_df

    # turn date from string type to datetime type
    aq_dataset["time"] = pd.to_datetime(aq_dataset['utc_time'])
    aq_dataset.set_index("time", inplace=True)
    aq_dataset.drop("utc_time", axis=1, inplace=True)

    aq_dataset = aq_dataset[pd.isnull(aq_dataset.stationId) != True]

    # names of all stations
    stations = set(aq_dataset['stationId'])
    # print(stations)

    # a dict of station aq
    aq_stations = {}
    for station in stations:
        aq_station = aq_dataset[aq_dataset["stationId"]==station].copy()
        aq_station.drop("stationId", axis=1, inplace=True)

        # rename
        original_names = aq_station.columns.values.tolist()
        names_dict = {original_name : station+"_"+original_name for original_name in original_names}
        aq_station.rename(index=str, columns=names_dict, inplace=True)
        aq_station.drop_duplicates(inplace=True)
        aq_stations[station] = aq_station
        # print(aq_station.shape)

    # for station in stations :
    # print(aq_stations[station].shape)

    # merge data of different stations into one df
    aq_stations_merged = pd.concat(list(aq_stations.values()), axis=1)
    # add a column of (0,1,2,3,...) to count
    length = aq_stations_merged.shape[0]
    order = range(length)
    aq_stations_merged['order'] = pd.Series(order, index=aq_stations_merged.index)

    return aq_dataset, stations, aq_stations, aq_stations_merged

In [11]:
# load the data again
airQuality = pd.read_csv('data/airQuality.csv')
airQuality.head()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin_aq,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin_aq,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin_aq,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin_aq,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin_aq,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0


In [12]:
bj_aq_data, stations, bj_aq_stations, bj_aq_stations_merged = load_bj_aq_data(airQuality)

In [13]:
print("最早的日期：", bj_aq_stations_merged.index.min())
print("最晚的日期：", bj_aq_stations_merged.index.max())
bj_aq_stations_merged.shape

最早的日期： 2017-01-01 14:00:00
最晚的日期： 2018-04-30 23:00:00


(10769, 211)

## 2.重复日期的值删除
遍历index, 找出对应的时间重复值，删除后出现的重复时间

In [14]:
df_merged = bj_aq_stations_merged
df_merged['time'] = df_merged.index
df_merged.set_index('order', inplace=True)

In [15]:
df_merged.head()

,gucheng_aq_PM2.5,gucheng_aq_PM10,gucheng_aq_NO2,gucheng_aq_CO,gucheng_aq_O3,gucheng_aq_SO2,tiantan_aq_PM2.5,tiantan_aq_PM10,tiantan_aq_NO2,tiantan_aq_CO,...,tongzhou_aq_CO,tongzhou_aq_O3,tongzhou_aq_SO2,dingling_aq_PM2.5,dingling_aq_PM10,dingling_aq_NO2,dingling_aq_CO,dingling_aq_O3,dingling_aq_SO2,time
order,,,,,,,,,,,,,,,,,,,,,
0,500.0,612.0,161.0,7.7,3.0,11.0,357.0,449.0,116.0,6.2,...,5.1,2.0,9.0,339.0,372.0,137.0,5.9,6.0,18.0,2017-01-01 14:00:00
1,490.0,594.0,150.0,7.9,3.0,11.0,351.0,467.0,109.0,6.4,...,5.4,3.0,10.0,343.0,NaN,137.0,5.4,4.0,19.0,2017-01-01 15:00:00
2,513.0,618.0,157.0,8.7,4.0,11.0,346.0,451.0,101.0,6.6,...,6.2,3.0,10.0,311.0,NaN,123.0,5.1,5.0,13.0,2017-01-01 16:00:00
3,528.0,609.0,170.0,9.9,4.0,13.0,339.0,447.0,98.0,6.6,...,6.5,4.0,12.0,256.0,NaN,118.0,4.1,3.0,17.0,2017-01-01 17:00:00
4,297.0,NaN,92.0,4.4,15.0,11.0,345.0,471.0,101.0,6.9,...,6.8,4.0,12.0,57.0,NaN,33.0,0.8,43.0,17.0,2017-01-01 18:00:00


In [17]:
print("去除重复数据前数据数量：", df_merged.shape[0])
df_merged = df_merged.reset_index()
df_merged.drop_duplicates(subset='time', keep='first', inplace=True)
print("去除重复数据后数据数量：", df_merged.shape[0])

去除重复数据前数据数量： 10769
去除重复数据后数据数量： 10769


In [18]:
df_merged.set_index('time', inplace=True)

## 3. 缺失值的分析
缺失分类：
（1）整体性缺失：某些时间点，所有站点的所有特征均缺失；
（2）个例性整体缺失：某些时间点，某个站点的所有数据缺失；
（3）个例性部分缺失：某个时间点，某个站点的部分数据缺失；

解决办法：
针对（1）：某天缺失超过5个小时，放弃使用改天数据，否则使用插值方法进行填充；
针对（2）（3）：使用距离该站有数据的最近一个站的数据，直接作为该站的数据。

In [20]:
min_time = df_merged.index.min()
max_time = df_merged.index.max()

min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')

delta_all = max_time-min_time
print("在空气质量数据时间段内，总共应该有%d个小时节点。" % (delta_all.total_seconds()/3600 +1))
print("实际的时间节点数是%d" % (df_merged.shape[0]))
print("缺失时间节点数量是%d" % (11626-10769))

在空气质量数据时间段内，总共应该有11626个小时节点。
实际的时间节点数是10769
缺失时间节点数量是857


##3.1 整体性缺失

统计哪些时间点发生了整小时的缺失

3.1 整体性缺失
统计哪些时间点发生了整小时的缺失

In [21]:
delta = datetime.timedelta(hours=1)
time = min_time
missing_hours=[]
missing_hours_str=[]

while time <= max_time:
    if datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S') not in df_merged.index:
        missing_hours.append(time)
        missing_hours_str.append(datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S'))
    time += delta

print("整体性缺失共计%d小时" % (len(missing_hours)))

整体性缺失共计857小时


3.2 个例性整体缺失：某些时间点，某个站点的所有数据缺失；

In [13]:
# using xlrd 、csv process .xlsx文件
from xlrd import open_workbook
import csv
import json
import queue

In [20]:
# load the 'Beijing_AirQuality_Stations_en.xlsx'信息，并且转存为.csv 
# stationID|latitude|logtitude|stationTYpe
wb = open_workbook('data/Beijing_AirQuality_Stations_en.xlsx')
sheet_names = wb.sheet_names()
sheet = wb.sheet_by_name(sheet_names[0])
number_of_rows = sheet.nrows
number_of_columns = sheet.ncols

stationId = []
longitude = []
latitude = []
stationType = []
current_type = ''
for row in range(11, number_of_rows):
    if(sheet.cell(row, 0).value == ''):
        continue
    elif(row in [12-1, 26-1, 39-1, 48-1]):
        current_type = sheet.cell(row, 0).value
    else:
        stationId.append(sheet.cell(row, 0).value)
        longitude.append(sheet.cell(row, 1).value)
        latitude.append(sheet.cell(row, 2).value)
        stationType.append(current_type)
D = {'stationId': pd.Series(stationId),
     'latitude': pd.Series(latitude),
     'longitude': pd.Series(longitude),
     'stationType': pd.Series(stationType),
    }

df_aq_station = pd.DataFrame(D)
df_aq_station.set_index('stationId', inplace=True)
df_aq_station.to_csv('data/Beijing_AirQuality_Stations_en.csv')
df_aq_station.head()

,latitude,longitude,stationType
stationId,,,
dongsi_aq,39.929,116.417,Urban Stations
tiantan_aq,39.886,116.407,Urban Stations
guanyuan_aq,39.929,116.339,Urban Stations
wanshouxigong_aq,39.878,116.352,Urban Stations
aotizhongxin_aq,39.982,116.397,Urban Stations


In [22]:
# 重新加载数据
aq_station_locations = pd.read_csv('data/Beijing_AirQuality_Stations_en.csv', index_col=False)
aq_station_locations.head()

,stationId,latitude,longitude,stationType
0,dongsi_aq,39.929,116.417,Urban Stations
1,tiantan_aq,39.886,116.407,Urban Stations
2,guanyuan_aq,39.929,116.339,Urban Stations
3,wanshouxigong_aq,39.878,116.352,Urban Stations
4,aotizhongxin_aq,39.982,116.397,Urban Stations


对于一个空气质量站点，将其他站点按照距该站点距离的大小关系排列，并保存为列表

In [23]:
for index_t in aq_station_locations.index:
    row_t = aq_station_locations.loc[index_t]
    # location of target station
    long_t = row_t['longitude']
    lati_t = row_t['latitude']
    # column name
    station_name = row_t['stationId']
    
    # add a new column to df
    all_dis = []
    for index in aq_station_locations.index:
        row = aq_station_locations.loc[index]
        long = row['longitude']
        lati = row['latitude']
        dis = np.sqrt((long-long_t)**2 + (lati-lati_t)**2)
        all_dis.append(dis)
        
    aq_station_locations[station_name] = all_dis

aq_station_locations

,stationId,latitude,longitude,stationType,dongsi_aq,tiantan_aq,guanyuan_aq,wanshouxigong_aq,aotizhongxin_aq,nongzhanguan_aq,...,miyunshuiku_aq,donggaocun_aq,yongledian_aq,yufa_aq,liulihe_aq,qianmen_aq,yongdingmennei_aq,xizhimenbei_aq,nansanhuan_aq,dongsihuan_aq
0,dongsi_aq,39.929,116.417,Urban Stations,0.000000,0.044147,0.078000,0.082620,0.056648,0.044721,...,0.754278,0.723498,0.425494,0.425406,0.543774,0.037202,0.057775,0.072450,0.087920,0.066753
1,tiantan_aq,39.886,116.407,Urban Stations,0.044147,0.000000,0.080455,0.055579,0.096519,0.074277,...,0.793590,0.744423,0.414309,0.381320,0.509200,0.017692,0.016401,0.089376,0.049204,0.092655
2,guanyuan_aq,39.929,116.339,Urban Stations,0.078000,0.080455,0.000000,0.052631,0.078568,0.122262,...,0.807517,0.799501,0.494191,0.410855,0.486541,0.063530,0.076381,0.026926,0.078549,0.144347
3,wanshouxigong_aq,39.878,116.352,Urban Stations,0.082620,0.055579,0.052631,0.000000,0.113318,0.123944,...,0.835537,0.799442,0.461863,0.361757,0.461203,0.047854,0.042048,0.076059,0.027203,0.144506
4,aotizhongxin_aq,39.982,116.397,Urban Stations,0.056648,0.096519,0.078568,0.113318,0.000000,0.078237,...,0.729030,0.732566,0.471058,0.472073,0.564989,0.083024,0.106042,0.055570,0.129294,0.096151
5,nongzhanguan_aq,39.937,116.461,Urban Stations,0.044721,0.074277,0.122262,0.123944,0.078237,0.000000,...,0.719961,0.678859,0.392822,0.447001,0.583069,0.076158,0.090609,0.113283,0.123329,0.022091
6,wanliu_aq,39.987,116.287,Urban Stations,0.142352,0.156847,0.077897,0.126909,0.110114,0.181041,...,0.807168,0.840630,0.567134,0.467181,0.498014,0.139313,0.154175,0.070235,0.154019,0.201792
7,beibuxinqu_aq,40.090,116.174,Urban Stations,0.291496,0.309685,0.230534,0.276818,0.247776,0.325235,...,0.842882,0.946053,0.716774,0.583760,0.538865,0.292099,0.306914,0.221633,0.303961,0.343922
8,zhiwuyuan_aq,40.002,116.207,Urban Stations,0.222326,0.231206,0.150841,0.190790,0.191050,0.262185,...,0.861757,0.918245,0.644884,0.490890,0.470035,0.214367,0.225488,0.149893,0.217341,0.283099
9,fengtaihuayuan_aq,39.863,116.279,Urban Stations,0.152971,0.130050,0.089196,0.074525,0.167586,0.196469,...,0.896616,0.873756,0.526134,0.343642,0.397404,0.121458,0.115732,0.114809,0.089275,0.217697


In [24]:
# 以每一个站的名字为key， 以其他站的名字组成的列表为value list， 列表中从前至后距离越来越远
near_stations = {}
for index_t in aq_station_locations.index:
    target_station_name = aq_station_locations.loc[index_t]['stationId']
    ordered_stations_names = aq_station_locations.sort_values(by=target_station_name)['stationId'].values[1:]
    near_stations[target_station_name] = ordered_stations_names

In [25]:
# 举个例子：dingling_aq 附近的、按照距离排序的站的名字
near_stations['dingling_aq']

array(['pingchang_aq', 'beibuxinqu_aq', 'badaling_aq', 'zhiwuyuan_aq',
       'yanqin_aq', 'wanliu_aq', 'aotizhongxin_aq', 'xizhimenbei_aq',
       'mentougou_aq', 'gucheng_aq', 'guanyuan_aq', 'huairou_aq',
       'dongsi_aq', 'nongzhanguan_aq', 'qianmen_aq', 'fengtaihuayuan_aq',
       'wanshouxigong_aq', 'dongsihuan_aq', 'tiantan_aq',
       'yongdingmennei_aq', 'nansanhuan_aq', 'shunyi_aq', 'yungang_aq',
       'fangshan_aq', 'yizhuang_aq', 'tongzhou_aq', 'daxing_aq',
       'miyun_aq', 'miyunshuiku_aq', 'liulihe_aq', 'yufa_aq',
       'yongledian_aq', 'pinggu_aq', 'donggaocun_aq'], dtype=object)

3.3 个别缺失值的处理

In [28]:
def get_estimated_value(station_name, feature_name, near_stations, row):
    #为缺失值feature 寻找替代
    near_stations = near_stations[station_name]
    # 在最近的站中依次寻找非缺失值，找到后进行替代
    for station in near_stations:
        feature = station+'_'+feature_name
        if not pd.isnull(row[feature]):
            return row[feature]
    return 0

In [45]:
for index in df_merged.index :
    row = df_merged.loc[index].copy()
    for feature in row.index :
#         print(feature)
        if pd.isnull(row[feature]) :
            elements = feature.split("_")                  # feature example： nansanhuan_aq_PM2.5
            station_name = elements[0] + "_" + elements[1] # nansanhuan_aq
            feature_name = elements[2]                     # PM2.5
            row[feature] = get_estimated_value(station_name, feature_name, near_stations, row)
    df_merged.loc[index] = row


In [46]:
# 用于查看现在是否还有缺失值
pd.isnull(df_merged).any().any()

False

In [47]:
# 查看插值之前的数据情况
print("插值之前数据维度：", df_merged.shape)

插值之前数据维度： (10769, 212)


In [54]:
del df_merged['index']
del df_merged['order']
df_merged.shape #(10769, 210)

(10769, 210)

3.4 整体性缺失情况的处理：
（1）如果连续缺失时长<=5小时，就进行补全；
（2）如果超过5小时，用NAN进行补全（整张表中唯一会出现NAN的情况）

In [55]:
# 根据连续缺失时长，将 missing hours 分成两类：keep hours | drop hours
keep_hours = []
drop_hours = []

In [58]:
# 针对情况（1） 进行补充
delta = datetime.timedelta(hours=1)

for hour in missing_hours_str : 
    
    time = datetime.datetime.strptime(hour, '%Y-%m-%d %H:%M:%S')
    
    # 前边第几个是非空的
    found_for = False
    i = 0
    while not found_for :
        i += 1
        for_time = time - i * delta
        for_time_str = datetime.date.strftime(for_time, '%Y-%m-%d %H:%M:%S')
        if for_time_str in df_merged.index :
            for_row = df_merged.loc[for_time_str]
            for_step = i
            found_for = True
            
            
    # 后边第几个是非空的
    found_back = False
    j = 0
    while not found_back :
        j += 1
        back_time = time + j * delta
        back_time_str = datetime.date.strftime(back_time, '%Y-%m-%d %H:%M:%S')
        if back_time_str in df_merged.index :
            back_row = df_merged.loc[back_time_str]
            back_step = j
            found_back = True
    
    # print(for_step, back_step)
    all_steps = for_step + back_step
    if all_steps > 5 :
        drop_hours.append(hour)
    else : 
        keep_hours.append(hour)
        # 插值
        delata_values = back_row - for_row
        df_merged.loc[hour] = for_row + (for_step/all_steps) * delata_values        

In [59]:
# 查看两种情况下的具体数量:drop_hours=621; keep_hours=236
print(len(drop_hours), len(keep_hours), len(missing_hours_str))

621 236 857


In [60]:
# 查看(1)插值之后的数据情况
print("插值之后数据维度：", df_merged.shape)#(11005, 210)

插值之后数据维度： (11005, 210)


In [61]:
# 用于查看现在是否还有缺失值
pd.isnull(df_merged).any().any()

False

In [62]:
# 针对情况（2）进行补充
nan_series = pd.Series({key:np.nan for key in df_merged.columns})

for hour in drop_hours:
    df_merged.loc[hour] = nan_series

df_merged.sort_index(inplace=True)

In [63]:
# 查看(1)(2)插值之后的数据情况
print("插值之后数据维度：", df_merged.shape) # (11626, 210)

插值之后数据维度： (11626, 210)


In [65]:
df_merged.to_csv('data/test/bj_aq_data.csv')

# 4. 数据归一化

In [66]:
describe = df_merged.describe()
describe.to_csv('data/bj_aq_describe.csv')

In [67]:
df_norm = (df_merged - describe.loc['mean'])/describe.loc['std']
df_norm.to_csv('data/bj_aq_norm_data.csv')